In [1]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#os.environ['PYSPARK_PYTHON'] = 'python2'

%matplotlib inline


# Featurize - same as in Spark.ipynb

In [2]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("train-features-predicted_mean.csv", index_col=['Date', 'Store'])
df_customers = DataFrame.from_csv("train-features-predicted_mean-customers.csv", index_col=['Date', 'Store'])
del df_means['Sales-prediction_mean_error'] # Not supported

In [3]:
df_sales = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'])[['Sales']]

/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df_features = df_means.join(df_basic, how='outer')
train_df = df_sales.join(df_features)
train_df = train_df.join(df_customers['Customers_predicted'])
train_df.head()

Sales  Sales_predicted  DayOfWeek  Open  Promo  \
Date       Store                                                   
2015-07-31 1       5263      5145.283582          5     1      1   
           2       6064      5115.882353          5     1      1   
           3       8314      8138.089552          5     1      1   
           4      13995     10275.776119          5     1      1   
           5       4822      5308.835821          5     1      1   

                  StateHoliday  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  \
Date       Store                                                               
2015-07-31 1                 0              1              31              4   
           2                 0              1              31              4   
           3                 0              1              31              4   
           4                 0              1              31              4   
           5                 0              1              31              4   

                  cal:dayofyear  cal:month  cal:quarter  cal:weekofyear  \
Date       Store                                                          
2015-07-31 1                212          7            3              31   
           2                212          7            3              31   
           3                212          7            3              31   
           4                212          7            3              31   
           5                212          7            3              31   

                  cal:year  Customers_predicted  
Date       Store                                 
2015-07-31 1          2015           588.194030  
           2          2015           600.441176  
           3          2015           838.223881  
           4          2015          1373.895522  
           5          2015           598.462687

In [5]:
del df_features
del df_basic
del df_means
del df_sales

In [6]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())
train_df['Sales_predicted'] = train_df['Sales_predicted'].map(lambda x: int(x))

In [7]:
train_df['Customers_predicted'] = train_df['Customers_predicted'].fillna(train_df['Sales_predicted'].mean())
train_df['Customers_predicted'] = train_df['Customers_predicted'].map(lambda x: int(x))

In [8]:
train_df.reset_index(inplace=True)
del train_df['Date']

In [9]:
feature_columns = list(train_df.columns)
feature_columns.remove("Sales")
feature_columns = ['Sales'] + feature_columns
train_df = train_df[feature_columns]

In [10]:

train_df.head()

,Sales,Store,Sales_predicted,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Customers_predicted
0,5263,1,5145,5,1,1,0,1,31,4,212,7,3,31,2015,588
1,6064,2,5115,5,1,1,0,1,31,4,212,7,3,31,2015,600
2,8314,3,8138,5,1,1,0,1,31,4,212,7,3,31,2015,838
3,13995,4,10275,5,1,1,0,1,31,4,212,7,3,31,2015,1373
4,4822,5,5308,5,1,1,0,1,31,4,212,7,3,31,2015,598


In [11]:
train_df, test_df = cross_validation.train_test_split(train_df, train_size=.90)

In [12]:
# Model

In [13]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [14]:
features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
regressor_big = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
regressor_big.fit(train_df[features], train_df['Sales'])
score(predicted=regressor_big.predict(test_df[features]), actual=test_df['Sales'])

0.14753300754198484

In [15]:
# Submit

In [20]:
df_basic = DataFrame.from_csv("test_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("test-features-predicted_mean.csv", index_col=['Date', 'Store'])

ValueError: Index Date invalid

In [ ]:
df_test_features = df_means.join(df_basic)

In [ ]:
df_test_features.fillna(0, inplace=True)
df_test_features['Sales_predicted'] = df_test_features['Sales_predicted'].map(lambda x: int(x))

In [ ]:
df_test_features.reset_index(inplace=True)
del df_test_features['Date']

In [17]:
df_test_features.set_index('Id', inplace=True)
df_test_features.head()

NameError: name 'df_test_features' is not defined

In [18]:

df_test_features.head()

NameError: name 'df_test_features' is not defined

In [19]:
len(df_test_features.columns)

NameError: name 'df_test_features' is not defined

In [ ]:

predictions = regressor_big.predict(df_test_features[features])

In [ ]:
df_test_features['Predictions'] = predictions

In [ ]:
score(df_test_features.Sales_predicted, df_test_features.Predictions) # BAd naming, compares mean prediction with ensemble prediction as sanity check

In [ ]:
ensemble_results = df_test_features[['Predictions']]
ensemble_results.columns=['Sales']
ensemble_results.to_csv( "results-ensemble.csv")

In [ ]:
 ensemble_results.head()